## Simple optimization model

In [5]:
import os
import numpy as np
from gurobipy import Model, GRB, quicksum
from Input_generator import generate_input

# ---------- user knobs ----------
N_list = list(range(100, 1001, 100))       # sample sizes
OUT_OF_SAMPLE_M = 2000         # fresh scenarios for OOS evaluation
SPILL_PENALTY = 1e-7           # tiny; discourages spill
LEVEL_PENALTY = 1e6            # big; strongly discourages level violations (only used in evaluation)
OUTPUT_DIR = "results_eev_ws"
SEED_OOS = 12345
# --------------------------------


In [6]:


def build_model(J, T, l_max, l_min, l0, pi_max, pi_min, R, prices_t, inflows_tj, g,
                soft_levels=False, level_penalty=0.0):
    """
    If soft_levels=True, bounds on l are enforced with slacks (>=0) and a penalty.
    Otherwise, l has hard bounds.
    """
    model = Model("hydro_revenue_max")
    model.Params.OutputFlag = 0
    Jset, Tset = range(J), range(T)

    # discharge
    pi = model.addVars(((j,t) for j in Jset for t in Tset),
                       lb={(j,t): float(pi_min[j]) for j in Jset for t in Tset},
                       ub={(j,t): float(pi_max[j]) for j in Jset for t in Tset},
                       name="pi")

    # reservoir levels
    if soft_levels:
        # no hard bounds; enforce with constraints + slacks
        l = model.addVars(((j,t) for j in Jset for t in range(T+1)),
                          lb=-GRB.INFINITY, name="l")
        l_ub_slack = model.addVars(((j,t) for j in Jset for t in range(T+1)), lb=0.0, name="l_ub_slack")
        l_lb_slack = model.addVars(((j,t) for j in Jset for t in range(T+1)), lb=0.0, name="l_lb_slack")
    else:
        l = model.addVars(((j,t) for j in Jset for t in range(T+1)),
                          lb={(j,t): float(l_min[j]) for j in Jset for t in range(T+1)},
                          ub={(j,t): float(l_max[j]) for j in Jset for t in range(T+1)},
                          name="l")
        l_ub_slack = l_lb_slack = None

    # spill
    s  = model.addVars(((j,t) for j in Jset for t in Tset), lb=0.0, name="spill")

    # initial levels
    for j in Jset:
        model.addConstr(l[j,0] == float(l0[j]))

    # dynamics: l_{j,t+1} = l_{j,t} + inflow_{t,j} + sum_k R[j,k]*pi_{k,t} - s_{j,t}
    for t in Tset:
        for j in Jset:
            model.addConstr(
                l[j,t+1] == l[j,t]
                + float(inflows_tj[t,j])
                + quicksum(float(R[j,k]) * pi[k,t] for k in Jset)
                - s[j,t]
            )

    # soft bounds if requested
    if soft_levels:
        for j in Jset:
            for t in range(T+1):
                model.addConstr(l[j,t] <= float(l_max[j]) + l_ub_slack[j,t])
                model.addConstr(l[j,t] >= float(l_min[j]) - l_lb_slack[j,t])

    # objective
    revenue = quicksum(float(prices_t[t]) * quicksum(float(g[j]) * pi[j,t] for j in Jset)
                       for t in Tset)
    obj = revenue - SPILL_PENALTY * quicksum(s[j,t] for j in Jset for t in Tset)
    if soft_levels:
        obj -= level_penalty * (quicksum(l_ub_slack[j,t] for j in Jset for t in range(T+1))
                                + quicksum(l_lb_slack[j,t] for j in Jset for t in range(T+1)))
    model.setObjective(obj, GRB.MAXIMIZE)
    return model, pi, l, s

def solve_ws_for_scenario(params, prices_t, inflows_tj):
    J,T,l_max,l_min,l0,pi_max,pi_min,R,g = params
    model,pi,l,s = build_model(J,T,l_max,l_min,l0,pi_max,pi_min,R,prices_t,inflows_tj,g,
                               soft_levels=False)
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return np.nan, None
    schedule = np.array([[pi[j,t].X for t in range(T)] for j in range(J)])
    return model.ObjVal, schedule

def evaluate_fixed_schedule(params, prices_t, inflows_tj, schedule):
    """
    Evaluate a fixed discharge schedule π on a scenario.
    Use soft level bounds with a large penalty to guarantee feasibility.
    """
    J,T,l_max,l_min,l0,pi_max,pi_min,R,g = params
    model,pi,l,s = build_model(J,T,l_max,l_min,l0,pi_max,pi_min,R,prices_t,inflows_tj,g,
                               soft_levels=True, level_penalty=LEVEL_PENALTY)
    for j in range(J):
        for t in range(T):
            model.addConstr(pi[j,t] == float(schedule[j,t]))
    model.optimize()
    if model.Status != GRB.OPTIMAL:
        return np.nan
    return model.ObjVal

def run_for_N(N):
    # In-sample data
    (J,T,l_max,l_min,l0,pi_max,pi_min,
     price_samples,inflow_samples,
     _nu0,_rho0,R,_a_t,_b_t,_l_bar,
     alpha_energy) = generate_input(N)

    params = (J,T,np.array(l_max),np.array(l_min),np.array(l0),
              np.array(pi_max),np.array(pi_min),np.array(R),np.array(alpha_energy))

    # EEV schedule from sample means
    prices_mean = np.mean(price_samples,axis=0)
    inflows_mean = np.mean(inflow_samples,axis=0)
    _, eev_schedule = solve_ws_for_scenario(params, prices_mean, inflows_mean)

    # In-sample evaluation
    eev_is, ws_is = [], []
    for s in range(N):
        obj_ws,_ = solve_ws_for_scenario(params, price_samples[s], inflow_samples[s])
        ws_is.append(obj_ws)
        obj_eev = evaluate_fixed_schedule(params, price_samples[s], inflow_samples[s], eev_schedule)
        eev_is.append(obj_eev)

    # Out-of-sample scenarios
    np.random.seed(SEED_OOS)
    (J_o, T_o,
     l_max_o, l_min_o, l0_o,
     pi_max_o, pi_min_o,
     price_oos, inflow_oos,
     _nu0_o, _rho0_o,
     R_o,
     _a_t_o, _b_t_o, _l_bar_o,
     alpha_energy_o) = generate_input(OUT_OF_SAMPLE_M)

    # reuse params (assuming same structure)
    params_o = params

    eev_oos, ws_oos = [], []
    for s in range(OUT_OF_SAMPLE_M):
        obj_ws,_ = solve_ws_for_scenario(params_o, price_oos[s], inflow_oos[s])
        ws_oos.append(obj_ws)
        obj_eev = evaluate_fixed_schedule(params_o, price_oos[s], inflow_oos[s], eev_schedule)
        eev_oos.append(obj_eev)

    # --------- Write CSVs ----------
    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Per-scenario (in-sample)
    with open(os.path.join(OUTPUT_DIR, f"in_sample_N{N}.csv"), "w") as f:
        f.write("scenario,WS,EEV,regret\n")
        for idx,(w,e) in enumerate(zip(ws_is,eev_is),1):
            wv = "" if np.isnan(w) else f"{w:.8f}"
            ev = "" if np.isnan(e) else f"{e:.8f}"
            rv = "" if (np.isnan(w) or np.isnan(e)) else f"{(w-e):.8f}"
            f.write(f"{idx},{wv},{ev},{rv}\n")

    # Per-scenario (out-of-sample)
    with open(os.path.join(OUTPUT_DIR, f"out_of_sample_N{N}.csv"), "w") as f:
        f.write("scenario,WS,EEV,regret\n")
        for idx,(w,e) in enumerate(zip(ws_oos,eev_oos),1):
            wv = "" if np.isnan(w) else f"{w:.8f}"
            ev = "" if np.isnan(e) else f"{e:.8f}"
            rv = "" if (np.isnan(w) or np.isnan(e)) else f"{(w-e):.8f}"
            f.write(f"{idx},{wv},{ev},{rv}\n")

    # EEV schedule as CSV (rows=time, cols=reservoirs)
    schedule_file = os.path.join(OUTPUT_DIR, f"eev_schedule_N{N}.csv")
    with open(schedule_file, "w") as f:
        header = ",".join([f"Reservoir{j+1}" for j in range(J)])
        f.write("time," + header + "\n")
        for t in range(T):
            row = ",".join(f"{eev_schedule[j,t]:.8f}" for j in range(J))
            f.write(f"{t+1},{row}\n")

def main():
    for N in N_list:
        run_for_N(N)

if __name__ == "__main__":
    main()
